In [68]:
import warnings
warnings.filterwarnings('ignore')

In [69]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [70]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [71]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [72]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
pd.isnull(df).sum()

# Split the Data into Training and Testing

In [ ]:
# # Create our features
# X = df.drop(target, axis=1)

# # Create our target
# y = df["loan_status"]


In [73]:
y = df[["loan_status"]]

# y = df["loan_status"].ravel()

In [74]:
# Create our features
X = df.drop(columns="loan_status", inplace=True)

In [75]:
# Encode all columns with dtype 'object' for ML processing.
X = pd.get_dummies(df, columns=list(
    df.select_dtypes(['object']).columns))

In [ ]:
y.describe()

In [ ]:
X.describe()

In [76]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [ ]:
y.value_counts()

In [ ]:
type(y)

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [78]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [79]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

In [80]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [81]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


In [82]:
# Calculated the balanced accuracy score
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [83]:
# List the features sorted in descending order by feature importance
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.0727639593858621, 'total_pymnt_inv'),
 (0.06834306477244799, 'last_pymnt_amnt'),
 (0.06782335661147988, 'total_rec_prncp'),
 (0.05739610678680039, 'total_pymnt'),
 (0.05720777169836709, 'total_rec_int'),
 (0.020688645276999505, 'out_prncp'),
 (0.02036466881582304, 'installment'),
 (0.018656936727152965, 'out_prncp_inv'),
 (0.017857917342116038, 'dti'),
 (0.01601229614899384, 'max_bal_bc'),
 (0.01574247260803747, 'loan_amnt'),
 (0.015731883401048953, 'mo_sin_old_rev_tl_op'),
 (0.015367475924442476, 'avg_cur_bal'),
 (0.014456619795496657, 'bc_open_to_buy'),
 (0.014130125299418065, 'revol_bal'),
 (0.013983344975541495, 'tot_hi_cred_lim'),
 (0.013963065564768017, 'mo_sin_old_il_acct'),
 (0.013865867169455131, 'total_bal_il'),
 (0.0138371824884218, 'tot_cur_bal'),
 (0.013826528564285607, 'bc_util'),
 (0.013665149058710115, 'total_bal_ex_mort'),
 (0.01360526986087437, 'total_il_high_credit_limit'),
 (0.013327468915488852, 'int_rate'),
 (0.01299461620448167, 'total_rev_hi_lim'),
 (0.01296

In [84]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


Accuracy Score : 0.9959895379250218
Classification Report
              precision    recall  f1-score   support

   high_risk       0.88      0.37      0.52       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.94      0.68      0.76     17205
weighted avg       1.00      1.00      1.00     17205



### Easy Ensemble AdaBoost Classifier

In [90]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=1)
   classifier.fit(X_train_scaled, y_train)

In [91]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, predictions)

In [92]:
# Display the confusion matrix
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

In [93]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.0727639593858621, 'total_pymnt_inv'),
 (0.06834306477244799, 'last_pymnt_amnt'),
 (0.06782335661147988, 'total_rec_prncp'),
 (0.05739610678680039, 'total_pymnt'),
 (0.05720777169836709, 'total_rec_int'),
 (0.020688645276999505, 'out_prncp'),
 (0.02036466881582304, 'installment'),
 (0.018656936727152965, 'out_prncp_inv'),
 (0.017857917342116038, 'dti'),
 (0.01601229614899384, 'max_bal_bc'),
 (0.01574247260803747, 'loan_amnt'),
 (0.015731883401048953, 'mo_sin_old_rev_tl_op'),
 (0.015367475924442476, 'avg_cur_bal'),
 (0.014456619795496657, 'bc_open_to_buy'),
 (0.014130125299418065, 'revol_bal'),
 (0.013983344975541495, 'tot_hi_cred_lim'),
 (0.013963065564768017, 'mo_sin_old_il_acct'),
 (0.013865867169455131, 'total_bal_il'),
 (0.0138371824884218, 'tot_cur_bal'),
 (0.013826528564285607, 'bc_util'),
 (0.013665149058710115, 'total_bal_ex_mort'),
 (0.01360526986087437, 'total_il_high_credit_limit'),
 (0.013327468915488852, 'int_rate'),
 (0.01299461620448167, 'total_rev_hi_lim'),
 (0.01296

In [94]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,37,64
Actual 1,5,17099


Accuracy Score : 0.9959895379250218
Classification Report
              precision    recall  f1-score   support

   high_risk       0.88      0.37      0.52       101
    low_risk       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.94      0.68      0.76     17205
weighted avg       1.00      1.00      1.00     17205

